# Enter Fisheries

In [1]:
from theano.sandbox import cuda
cuda.use('gpu1')


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [52]:
%cd /home/ubuntu/nbs/lesson4
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

/home/ubuntu/nbs/lesson4


In [3]:
batch_size = 64

## Setup batches 

To do:
1. create validation set
2. create sample set
2. mode to separate dirs for each set
3. finetune and train

In [4]:
from __future__ import division,print_function
import math
import cPickle as pickle
import os, json
from glob import glob
import numpy as np
from matplotlib import pyplot as plt

import PIL
from PIL import Image
#import cv2
from numpy.random import random, permutation, randn, normal, uniform
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.ndimage import imread
from sklearn.metrics import confusion_matrix
import bcolz
import itertools

In [5]:
%cd "/home/ubuntu/data/fisheries"

/home/ubuntu/data/fisheries


In [6]:
%cd train

/home/ubuntu/data/fisheries/train


### create validation set

In [7]:
%mkdir ../valid

mkdir: cannot create directory ‘../valid’: File exists


In [8]:
for d in glob('*'):
    print(d)
    os.mkdir('../valid/'+d)

DOL


OSError: [Errno 17] File exists: '../valid/DOL'

In [ ]:
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(400): os.rename(shuf[i], '../valid/'+shuf[i])

In [9]:
%ls -1 */*.jpg | cut -f1 -d"/" | sort | uniq -c

   1551 ALB
    183 BET
    105 DOL
     56 LAG
    408 NoF
    259 OTHER
    158 SHARK
    656 YFT
      1 ALB


In [10]:
 ls -1 ../valid/*/*.jpg | cut -f3 -d"/" | sort | uniq -c

    167 ALB
     17 BET
     12 DOL
     11 LAG
     57 NoF
     40 OTHER
     18 SHARK
     78 YFT


###  create sample

In [ ]:
from shutil import copyfile

In [ ]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [ ]:
for d in glob('*'):
    print(d)
    os.mkdir('../sample/valid/'+d)
    os.mkdir('../sample/train/'+d)

In [ ]:
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(400): copyfile(shuf[i], '../sample/train/'+shuf[i])

In [11]:
 ls -1 ../sample/train/*/*.jpg | cut -f4 -d"/" | sort | uniq -c

    190 ALB
     13 BET
     12 DOL
      4 LAG
     50 NoF
     34 OTHER
     20 SHARK
     77 YFT


In [ ]:
%cd ../valid
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(100): copyfile(shuf[i], '../sample/valid/'+shuf[i])

In [12]:
 ls -1 ../sample/valid/*/*.jpg | cut -f4 -d"/" | sort | uniq -c

     42 ALB
      1 BET
      4 DOL
      3 LAG
     23 NoF
      8 OTHER
      1 SHARK
     18 YFT


### Create batches

In [14]:
path = '/home/ubuntu/data/fisheries/'
batches = get_batches(path+'train', batch_size=batch_size)

Found 3377 images belonging to 8 classes.


In [15]:
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 400 images belonging to 8 classes.


In [16]:
test_batches = get_batches(path+'test', batch_size=batch_size*2, shuffle=False)

Found 1000 images belonging to 1 classes.


### Imagenet conv features

- Small data set similar to imagenet images (full color photos)
- Use pre-trained VGG weights
- Precompute the output of the last convolutional layer

In [17]:
vgg = Vgg16()
model = vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [18]:
conv_model = Sequential(conv_layers)

In [19]:
(val_classes, trn_classes, val_labels, trn_labels) = get_classes(path)

Found 3377 images belonging to 8 classes.
Found 400 images belonging to 8 classes.


In [20]:
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)


OSError: [Errno 2] No such file or directory: '/home/ubuntu/data/fisheries/results/conv_val_feat.dat'

In [21]:
%mkdir /home/ubuntu/data/fisheries/results

In [22]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [23]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_val_feat.shape

(400, 512, 14, 14)

### Batchnorm dense layers on pretrained conv layers

In [53]:
def get_maxpool_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(8, activation='softmax')
        ]

In [131]:
def get_avgpool_bn_layers(p):
    return [
        AveragePooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(8, activation='softmax')
        ]

In [132]:
p = 0.8

In [56]:
bn_mp_model = Sequential(get_maxpool_bn_layers(p))

In [57]:
bn_mp_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [58]:
print(conv_feat.shape)
print(conv_val_feat.shape)
print(trn_labels.shape)
print(val_labels.shape)

(3377, 512, 14, 14)
(400, 512, 14, 14)
(3377, 8)
(400, 8)


In [60]:
bn_mp_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=10, 
             validation_data=(conv_val_feat, val_labels))

Train on 3377 samples, validate on 400 samples
Epoch 1/10
3377/3377 [==============================] - 1s - loss: 3.3323 - acc: 0.2005 - val_loss: 2.3482 - val_acc: 0.2275
Epoch 2/10
3377/3377 [==============================] - 1s - loss: 2.7578 - acc: 0.2701 - val_loss: 1.9607 - val_acc: 0.3550
Epoch 3/10
3377/3377 [==============================] - 1s - loss: 2.4693 - acc: 0.3003 - val_loss: 1.8229 - val_acc: 0.3925
Epoch 4/10
3377/3377 [==============================] - 1s - loss: 2.2200 - acc: 0.3367 - val_loss: 1.8728 - val_acc: 0.3525
Epoch 5/10
3377/3377 [==============================] - 1s - loss: 2.1112 - acc: 0.3550 - val_loss: 1.8612 - val_acc: 0.3900
Epoch 6/10
3377/3377 [==============================] - 1s - loss: 2.0193 - acc: 0.3752 - val_loss: 1.8514 - val_acc: 0.4000
Epoch 7/10
3377/3377 [==============================] - 1s - loss: 1.8975 - acc: 0.3906 - val_loss: 1.8119 - val_acc: 0.4025
Epoch 8/10
3377/3377 [==============================] - 1s - loss: 1.8482 - ac

In [49]:
bn_mp_model.save_weights(path + 'models/bn_mp_epoch1.h5')

In [120]:
from keras.layers.convolutional import AveragePooling2D
bn_ap_model = Sequential(get_avgpool_bn_layers(p))

In [121]:
bn_ap_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [122]:
bn_ap_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=10, 
             validation_data=(conv_val_feat, val_labels))

Train on 3377 samples, validate on 400 samples
Epoch 1/10
3377/3377 [==============================] - 1s - loss: 2.9269 - acc: 0.1685 - val_loss: 2.1919 - val_acc: 0.2825
Epoch 2/10
3377/3377 [==============================] - 1s - loss: 2.2909 - acc: 0.2550 - val_loss: 1.9376 - val_acc: 0.3700
Epoch 3/10
3377/3377 [==============================] - 1s - loss: 2.0474 - acc: 0.3364 - val_loss: 1.9123 - val_acc: 0.3950
Epoch 4/10
3377/3377 [==============================] - 1s - loss: 1.8870 - acc: 0.3728 - val_loss: 1.8241 - val_acc: 0.4225
Epoch 5/10
3377/3377 [==============================] - 1s - loss: 1.7467 - acc: 0.4134 - val_loss: 1.8295 - val_acc: 0.3975
Epoch 6/10
3377/3377 [==============================] - 1s - loss: 1.7038 - acc: 0.4229 - val_loss: 1.8779 - val_acc: 0.3825
Epoch 7/10
3377/3377 [==============================] - 1s - loss: 1.6370 - acc: 0.4424 - val_loss: 1.8631 - val_acc: 0.4100
Epoch 8/10
3377/3377 [==============================] - 1s - loss: 1.5724 - ac

### Pre-computed data augmentation + dropout

In [50]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

Found 3377 images belonging to 8 classes.


Use image data generator to create a dataset of conv features 5x larger

In [51]:
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*5)

In [70]:
save_array(path+'results/da_conv_feat.dat', da_conv_feat)

In [ ]:
Combine real training data with augmented data

In [71]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])
da_trn_labels = np.concatenate([trn_labels]*6)

In [82]:
bn_mp_da_model = Sequential(get_maxpool_bn_layers(p))
bn_mp_da_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [83]:
bn_mp_da_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

Train on 20262 samples, validate on 400 samples
Epoch 1/4
20262/20262 [==============================] - 8s - loss: 3.4739 - acc: 0.2554 - val_loss: 0.8956 - val_acc: 0.7725
Epoch 2/4
20262/20262 [==============================] - 8s - loss: 2.2967 - acc: 0.4430 - val_loss: 0.6251 - val_acc: 0.8400
Epoch 3/4
20262/20262 [==============================] - 8s - loss: 1.7767 - acc: 0.5556 - val_loss: 0.5871 - val_acc: 0.8625
Epoch 4/4
20262/20262 [==============================] - 9s - loss: 1.4659 - acc: 0.6341 - val_loss: 0.6163 - val_acc: 0.8500


In [141]:
bn_ap_da_model = Sequential(get_avgpool_bn_layers(p))
bn_ap_da_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [142]:
bn_ap_da_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, val_labels))

Train on 20262 samples, validate on 400 samples
Epoch 1/2
20262/20262 [==============================] - 10s - loss: 3.2813 - acc: 0.3036 - val_loss: 0.7090 - val_acc: 0.8150
Epoch 2/2
20262/20262 [==============================] - 9s - loss: 2.0364 - acc: 0.5215 - val_loss: 0.5015 - val_acc: 0.8575


### Test and submit predictions from bn_mp_da, bn_ap_da models

In [86]:
mp_preds = bn_mp_da_model.predict(conv_test_feat, batch_size=batch_size*2)

In [143]:
ap_preds = bn_ap_da_model.predict(conv_test_feat, batch_size=batch_size*2)

In [144]:
ap_preds.shape

(1000, 8)

In [145]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [146]:
classes

['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [147]:
test_filenames = test_batches.filenames

In [95]:
test_filenames[1:10]

['unknown/img_06893.jpg',
 'unknown/img_02082.jpg',
 'unknown/img_06261.jpg',
 'unknown/img_03628.jpg',
 'unknown/img_04860.jpg',
 'unknown/img_01640.jpg',
 'unknown/img_02709.jpg',
 'unknown/img_00030.jpg',
 'unknown/img_04740.jpg']

In [109]:
import pandas as pd
ap_submit = pd.DataFrame(ap_preds, columns=classes)
ap_submit.insert(0, 'image', [a[8:] for a in test_filenames])

In [110]:
ap_submit.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_06237.jpg,0.355891,0.040566,0.059308,0.026525,0.063588,0.037628,0.031134,0.385359
1,img_06893.jpg,0.603492,0.033006,0.034771,0.018863,0.082407,0.037342,0.027252,0.162867
2,img_02082.jpg,0.198528,0.037253,0.044142,0.027116,0.411799,0.035010,0.124108,0.122044
3,img_06261.jpg,0.510785,0.032529,0.021745,0.018666,0.066152,0.039928,0.023479,0.286716
4,img_03628.jpg,0.621451,0.022591,0.033991,0.020155,0.062703,0.053016,0.032269,0.153825


In [111]:
ap_submit_name = path+'results/ap_submit.csv'
ap_submit.to_csv(ap_submit_name, index=False)
print(path)

/home/ubuntu/data/fisheries/


In [112]:
FileLink('../../data/fisheries/results/ap_submit.csv')

/home/ubuntu/data/fisheries/results/ap_submit.csv

In [113]:
mp_submit = pd.DataFrame(mp_preds, columns=classes)
mp_submit.insert(0, 'image', [a[8:] for a in test_filenames])

In [114]:
mp_submit.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_06237.jpg,0.362001,0.035319,0.020658,0.040183,0.062452,0.052852,0.026007,0.400528
1,img_06893.jpg,0.567150,0.033538,0.032001,0.027656,0.055870,0.068604,0.041292,0.173889
2,img_02082.jpg,0.487445,0.016200,0.013801,0.031727,0.176708,0.028245,0.114046,0.131829
3,img_06261.jpg,0.644842,0.018643,0.011207,0.014032,0.023648,0.038594,0.014768,0.234266
4,img_03628.jpg,0.749803,0.013427,0.015827,0.013549,0.027336,0.044968,0.014604,0.120486


In [115]:
mp_submit_name = path+'results/mp_submit.csv'
mp_submit.to_csv(mp_submit_name, index=False)
print(path)

/home/ubuntu/data/fisheries/


In [116]:
FileLink('../../data/fisheries/results/mp_submit.csv')

/home/ubuntu/data/fisheries/results/mp_submit.csv